# Exponent Closeness Between Black Holes

**Question:** Do the 13 black holes share the same bfloat16 exponent on all 2,560 dimensions?

If yes: the lattice is uniform across all black holes, and we can compare them directly in ULP.

If no: different holes live in different exponent regimes, and distance comparisons get complicated.

---

In [1]:
import torch
from safetensors.torch import load_file
import itertools

In [2]:
# Load our artifacts
tensors = load_file("../tensors/qwen3_4b_instruct_2507.safetensors")
centroids = tensors["black_hole_centroids"]  # [13, 2560] bf16

print(f"Centroids shape: {centroids.shape}")
print(f"Centroids dtype: {centroids.dtype}")

Centroids shape: torch.Size([13, 2560])
Centroids dtype: torch.bfloat16


## Extract Exponents

For bfloat16, the exponent is bits 7-14 (8 bits). We extract it by viewing as uint16 and bit-shifting.

In [3]:
def extract_bf16_exponent(tensor_bf16):
    """Extract the 8-bit exponent from each bfloat16 value."""
    bits = tensor_bf16.view(torch.uint16).to(torch.int32)
    exponent = (bits >> 7) & 0xFF
    return exponent

# Extract exponents for all centroids: [13, 2560]
exponents = extract_bf16_exponent(centroids)
print(f"Exponents shape: {exponents.shape}")
print(f"Exponent range: [{exponents.min().item()}, {exponents.max().item()}]")

Exponents shape: torch.Size([13, 2560])
Exponent range: [107, 123]


## Check: Do All Black Holes Share the Same Exponent on Every Axis?

In [4]:
# For each dimension, are all 13 exponents the same?
# Check if max == min along the black-hole axis
exp_min = exponents.min(dim=0).values  # [2560]
exp_max = exponents.max(dim=0).values  # [2560]

same_exponent_per_dim = (exp_min == exp_max)
num_same = same_exponent_per_dim.sum().item()
num_diff = (~same_exponent_per_dim).numel() - num_same

print(f"Dimensions where all 13 black holes share the same exponent: {num_same:,}")
print(f"Dimensions where exponents differ: {num_diff:,}")

Dimensions where all 13 black holes share the same exponent: 2,560
Dimensions where exponents differ: 0


## Pairwise Analysis

For each pair of black holes, how many of the 2,560 dimensions share the same exponent?

In [5]:
num_holes = centroids.shape[0]
D = centroids.shape[1]

print(f"Pairwise exponent agreement (out of {D:,} dimensions):\n")

# Build a matrix
agreement_matrix = torch.zeros(num_holes, num_holes, dtype=torch.int32)

for i, j in itertools.combinations(range(num_holes), 2):
    same = (exponents[i] == exponents[j]).sum().item()
    agreement_matrix[i, j] = same
    agreement_matrix[j, i] = same

# Fill diagonal with D (self-agreement is perfect)
for i in range(num_holes):
    agreement_matrix[i, i] = D

# Print as a table
print("    ", end="")
for j in range(num_holes):
    print(f"  BH{j+1:02d}", end="")
print()

for i in range(num_holes):
    print(f"BH{i+1:02d}", end="")
    for j in range(num_holes):
        val = agreement_matrix[i, j].item()
        if i == j:
            print("     -", end="")
        else:
            print(f"  {val:4d}", end="")
    print()

Pairwise exponent agreement (out of 2,560 dimensions):

      BH01  BH02  BH03  BH04  BH05  BH06  BH07  BH08  BH09  BH10  BH11  BH12  BH13
BH01     -  2560  2560  2560  2560  2560  2560  2560  2560  2560  2560  2560  2560
BH02  2560     -  2560  2560  2560  2560  2560  2560  2560  2560  2560  2560  2560
BH03  2560  2560     -  2560  2560  2560  2560  2560  2560  2560  2560  2560  2560
BH04  2560  2560  2560     -  2560  2560  2560  2560  2560  2560  2560  2560  2560
BH05  2560  2560  2560  2560     -  2560  2560  2560  2560  2560  2560  2560  2560
BH06  2560  2560  2560  2560  2560     -  2560  2560  2560  2560  2560  2560  2560
BH07  2560  2560  2560  2560  2560  2560     -  2560  2560  2560  2560  2560  2560
BH08  2560  2560  2560  2560  2560  2560  2560     -  2560  2560  2560  2560  2560
BH09  2560  2560  2560  2560  2560  2560  2560  2560     -  2560  2560  2560  2560
BH10  2560  2560  2560  2560  2560  2560  2560  2560  2560     -  2560  2560  2560
BH11  2560  2560  2560  2560  2

In [6]:
# Summary stats
# Get just the upper triangle (excluding diagonal)
upper_tri = agreement_matrix[torch.triu(torch.ones(num_holes, num_holes), diagonal=1) == 1]

print(f"\nPairwise agreement statistics:")
print(f"  Min:  {upper_tri.min().item():,} / {D:,} ({100*upper_tri.min().item()/D:.1f}%)")
print(f"  Max:  {upper_tri.max().item():,} / {D:,} ({100*upper_tri.max().item()/D:.1f}%)")
print(f"  Mean: {upper_tri.float().mean().item():,.1f} / {D:,} ({100*upper_tri.float().mean().item()/D:.1f}%)")


Pairwise agreement statistics:
  Min:  2,560 / 2,560 (100.0%)
  Max:  2,560 / 2,560 (100.0%)
  Mean: 2,560.0 / 2,560 (100.0%)


## Verdict

In [7]:
if num_same == D:
    print("✓ ALL dimensions share the same exponent across ALL black holes.")
    print("  The lattice is uniform. We can compare in ULP directly.")
else:
    print(f"✗ {num_diff} dimensions have varying exponents across black holes.")
    print("  The lattice is NOT uniform. Distance comparisons need care.")

✓ ALL dimensions share the same exponent across ALL black holes.
  The lattice is uniform. We can compare in ULP directly.
